<a href="https://colab.research.google.com/github/floydpip/BMR-Project/blob/computer_vision/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

# Computer vision

First we will start by importing the live video of the arena. We will retrieve that last element in the video array for processing.

Reference:
 https://towardsdatascience.com/video-streaming-in-the-jupyter-notebook-635bc5809e85


# Localization

# Algorithm for movement